<a href="https://colab.research.google.com/github/Chuc-ngan/Low-Light-Image-Enhancement/blob/main/Retinex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow==2.18.0  # Thay đổi phiên bản theo nhu cầu

# **1. Kết nối với Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd '/content/gdrive/MyDrive/ColabNotebooks/TieuLuan'

!pwd

Mounted at /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan
/content/gdrive/.shortcut-targets-by-id/1DnsWp1mopgehQJUYpN4ERfnj2CyapJGW/ColabNotebooks/TieuLuan


# **2. Import thư viện**

In [ ]:
# ===================== 1. Xử lý dữ liệu =====================
import os
import shutil  # Quản lý tệp/thư mục
import random
import math
import collections
import numpy as np  # Xử lý số học
import pandas as pd  # Xử lý dữ liệu CSV
from pathlib import Path
from PIL import Image, UnidentifiedImageError
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import Sequence
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tqdm import tqdm
import pickle
# ===================== 2. Machine Learning =====================
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import seaborn as sns

# ===================== 3. TensorFlow & Keras =====================
import tensorflow as tf
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import mixed_precision
from tensorflow.keras.regularizers import l2
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

# ===================== 4. Xây dựng mô hình =====================
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization,
    GlobalAveragePooling2D, Dense, Dropout, RepeatVector, Reshape, Flatten
)
from tensorflow.keras.applications import EfficientNetB0, ResNet50
from keras.applications.inception_resnet_v2 import InceptionResNetV2, preprocess_input, decode_predictions
from keras.layers import add, concatenate
from keras import layers, backend as K

# ===================== 5. Xử lý hình ảnh & Vẽ biểu đồ =====================
import cv2 as cv
import matplotlib.pyplot as plt

# ===================== 6. Đặt Seed (Reproducibility) =====================
tf.random.set_seed(42)  # Thay số 42 bằng giá trị seed mong muốn
np.random.seed(1)


# **3. Cấu hình môi trường TensorFlow**

In [ ]:
# Thiết lập GPU sử dụng bộ nhớ động (tránh lỗi thiếu bộ nhớ)
gpu_devices = tf.config.list_physical_devices('GPU')
if gpu_devices:
    tf.config.experimental.set_memory_growth(gpu_devices[0], True)

# Kiểm tra TensorFlow đang chạy trên GPU hay không
print("TensorFlow is running on GPU:", tf.test.is_built_with_cuda())
# Bật chế độ Mixed Precision để tăng tốc (nếu GPU hỗ trợ)
# Thiết lập mixed precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# Bật XLA với cấu hình tối ưu
tf.config.optimizer.set_jit(True)
tf.config.optimizer.set_experimental_options({
    "layout_optimizer": True,
    "constant_folding": True,
    "shape_optimization": True,
    "remapping": True,
    "arithmetic_optimization": True,
    "dependency_optimization": True,
    "loop_optimization": True,
    "function_optimization": True,
    "debug_stripper": True
})

TensorFlow is running on GPU: True


# **4. Configuration**

In [ ]:
# Thư mục thiếu sáng
low_light_dir = "dataset/low_light"

low_light_noise_dir = "dataset/low_light_noise"

# Thư mục chứa ảnh sau khi tăng cường
enhanced_image_low_light_dir = "dataset/enhanced_retinex_low_light"
enhanced_image_low_light_noise_dir = "dataset/enhanced_retinex_low_light_noise"

In [ ]:
def single_scale_retinex(img, sigma):
    blur = cv.GaussianBlur(img, (0, 0), sigma)
    return np.log1p(img) - np.log1p(blur + 1e-6)

def multi_scale_retinex(img, sigma_list):
    retinex = np.zeros_like(img)
    for sigma in sigma_list:
        retinex += single_scale_retinex(img, sigma)
    retinex /= len(sigma_list)
    return retinex

def color_restoration(img, alpha=125, beta=46):
    img_sum = np.sum(img, axis=2, keepdims=True)
    color_rest = beta * (np.log1p(alpha * img) - np.log1p(img_sum + 1e-6))
    return color_rest

def msrcr(img, sigma_list=[15, 80, 250], gain=1.5, offset=30):
    img = img.astype(np.float32) + 1.0  # tránh log(0)
    retinex = multi_scale_retinex(img, sigma_list)
    color_rest = color_restoration(img / 255.0)
    msrcr_result = gain * (retinex * color_rest)

    # Chuẩn hóa lại giá trị pixel về 0–255
    for i in range(msrcr_result.shape[2]):
        channel = msrcr_result[:, :, i]
        channel = cv.normalize(channel, None, 0, 255, cv.NORM_MINMAX)
        msrcr_result[:, :, i] = channel

    return np.clip(msrcr_result, 0, 255).astype(np.uint8)

In [ ]:
def enhance_msrcr_images(input_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)

    all_images = []
    for root, _, files in os.walk(input_dir):
        rel_path = os.path.relpath(root, input_dir)
        target_dir = os.path.join(output_dir, rel_path)
        os.makedirs(target_dir, exist_ok=True)

        image_paths = [f for f in files if f.lower().endswith(('.jpeg', '.png','.jpg'))]
        for file in image_paths:
            all_images.append((os.path.join(root, file), os.path.join(target_dir, file)))
    for input_path, output_path in tqdm(all_images, desc="MSRCR Enhancing", dynamic_ncols=True):
        img = cv.imread(input_path)
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        enhanced = msrcr(img)
        enhanced_bgr = cv.cvtColor(enhanced, cv.COLOR_RGB2BGR)
        cv.imwrite(output_path, enhanced_bgr)

    print(f"✅ Đã xử lý {len(all_images)} ảnh và lưu vào: {output_dir}")

In [ ]:
enhance_msrcr_images(low_light_dir, enhanced_image_low_light_dir)

MSRCR Enhancing: 100%|██████████| 10402/10402 [2:12:50<00:00,  1.31it/s]

✅ Đã xử lý 10402 ảnh và lưu vào: dataset/enhanced_retinex_low_light


In [ ]:
enhance_msrcr_images(low_light_noise_dir, enhanced_image_low_light_noise_dir)

MSRCR Enhancing: 100%|██████████| 10402/10402 [1:17:21<00:00,  2.24it/s]

✅ Đã xử lý 10402 ảnh và lưu vào: dataset/enhanced_retinex_low_light_noise
